# SwinIR 图像复原模型下载和测试

本notebook用于在AutoDL云端环境下载SwinIR预训练模型并进行各种图像复原任务的测试。

## 功能包括：
- 📥 自动下载预训练模型
- 🖼️ 图像超分辨率（2x, 4x, 8x）
- 🔧 图像去噪（灰度/彩色）
- 📺 JPEG压缩伪影减少
- 📊 结果可视化和质量评估

---

## 1. 环境设置和依赖安装

首先检查GPU可用性并安装必要的依赖包。

In [5]:
import os
import sys
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
import subprocess
from pathlib import Path

# 检查GPU可用性
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU数量: {torch.cuda.device_count()}")
    print(f"当前GPU: {torch.cuda.current_device()}")
    print(f"GPU名称: {torch.cuda.get_device_name()}")

# 设置工作目录
os.chdir('/root/SwinIR')
print(f"当前工作目录: {os.getcwd()}")

# 检查必要的目录结构
required_dirs = ['model_zoo/swinir', 'testsets', 'results', 'models', 'utils']
for dir_name in required_dirs:
    if not os.path.exists(dir_name):
        os.makedirs(dir_name, exist_ok=True)
        print(f"创建目录: {dir_name}")
    else:
        print(f"目录已存在: {dir_name}")

print("\n✅ 环境设置完成！")

PyTorch版本: 1.10.0+cu113
CUDA可用: True
GPU数量: 1
当前GPU: 0
GPU名称: NVIDIA GeForce RTX 4090
当前工作目录: /root/SwinIR
目录已存在: model_zoo/swinir
目录已存在: testsets
目录已存在: results
目录已存在: models
目录已存在: utils

✅ 环境设置完成！


## 2. 下载预训练模型

使用wget命令从GitHub releases下载SwinIR的所有预训练模型。这里提供了所有任务的模型文件。

In [6]:
# SwinIR预训练模型下载链接
models = {
    # 经典图像超分辨率模型
    "001_classicalSR_DIV2K_s48w8_SwinIR-M_x2.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/001_classicalSR_DIV2K_s48w8_SwinIR-M_x2.pth",
    "001_classicalSR_DIV2K_s48w8_SwinIR-M_x3.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/001_classicalSR_DIV2K_s48w8_SwinIR-M_x3.pth",
    "001_classicalSR_DIV2K_s48w8_SwinIR-M_x4.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/001_classicalSR_DIV2K_s48w8_SwinIR-M_x4.pth",
    "001_classicalSR_DIV2K_s48w8_SwinIR-M_x8.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/001_classicalSR_DIV2K_s48w8_SwinIR-M_x8.pth",
    "001_classicalSR_DF2K_s64w8_SwinIR-M_x2.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/001_classicalSR_DF2K_s64w8_SwinIR-M_x2.pth",
    "001_classicalSR_DF2K_s64w8_SwinIR-M_x3.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/001_classicalSR_DF2K_s64w8_SwinIR-M_x3.pth",
    "001_classicalSR_DF2K_s64w8_SwinIR-M_x4.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/001_classicalSR_DF2K_s64w8_SwinIR-M_x4.pth",
    "001_classicalSR_DF2K_s64w8_SwinIR-M_x8.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/001_classicalSR_DF2K_s64w8_SwinIR-M_x8.pth",
    
    # 轻量级超分辨率模型
    "002_lightweightSR_DIV2K_s64w8_SwinIR-S_x2.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/002_lightweightSR_DIV2K_s64w8_SwinIR-S_x2.pth",
    "002_lightweightSR_DIV2K_s64w8_SwinIR-S_x3.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/002_lightweightSR_DIV2K_s64w8_SwinIR-S_x3.pth",
    "002_lightweightSR_DIV2K_s64w8_SwinIR-S_x4.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/002_lightweightSR_DIV2K_s64w8_SwinIR-S_x4.pth",
    
    # 真实世界超分辨率模型
    "003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth",
    "003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth",
    
    # 灰度图像去噪模型
    "004_grayDN_DFWB_s128w8_SwinIR-M_noise15.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/004_grayDN_DFWB_s128w8_SwinIR-M_noise15.pth",
    "004_grayDN_DFWB_s128w8_SwinIR-M_noise25.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/004_grayDN_DFWB_s128w8_SwinIR-M_noise25.pth",
    "004_grayDN_DFWB_s128w8_SwinIR-M_noise50.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/004_grayDN_DFWB_s128w8_SwinIR-M_noise50.pth",
    
    # 彩色图像去噪模型
    "005_colorDN_DFWB_s128w8_SwinIR-M_noise15.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/005_colorDN_DFWB_s128w8_SwinIR-M_noise15.pth",
    "005_colorDN_DFWB_s128w8_SwinIR-M_noise25.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/005_colorDN_DFWB_s128w8_SwinIR-M_noise25.pth",
    "005_colorDN_DFWB_s128w8_SwinIR-M_noise50.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/005_colorDN_DFWB_s128w8_SwinIR-M_noise50.pth",
    
    # JPEG压缩伪影减少模型
    "006_CAR_DFWB_s126w7_SwinIR-M_jpeg10.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/006_CAR_DFWB_s126w7_SwinIR-M_jpeg10.pth",
    "006_CAR_DFWB_s126w7_SwinIR-M_jpeg20.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/006_CAR_DFWB_s126w7_SwinIR-M_jpeg20.pth",
    "006_CAR_DFWB_s126w7_SwinIR-M_jpeg30.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/006_CAR_DFWB_s126w7_SwinIR-M_jpeg30.pth",
    "006_CAR_DFWB_s126w7_SwinIR-M_jpeg40.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/006_CAR_DFWB_s126w7_SwinIR-M_jpeg40.pth",
    
    # 彩色JPEG压缩伪影减少模型
    "006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg10.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg10.pth",
    "006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg20.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg20.pth",
    "006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg30.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg30.pth",
    "006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg40.pth": "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg40.pth",
}

print(f"总共需要下载 {len(models)} 个模型文件")

总共需要下载 27 个模型文件


In [7]:
def download_model(model_name, url, force_download=False):
    """使用wget下载模型文件"""
    model_path = f"model_zoo/swinir/{model_name}"
    
    # 检查文件是否已存在
    if os.path.exists(model_path) and not force_download:
        print(f"✅ {model_name} 已存在，跳过下载")
        return True
    
    print(f"📥 开始下载 {model_name}...")
    try:
        # 使用wget下载，支持断点续传
        cmd = f"wget --no-check-certificate -c -O {model_path} {url}"
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        
        if result.returncode == 0:
            # 验证文件大小
            if os.path.exists(model_path) and os.path.getsize(model_path) > 1024*1024:  # 至少1MB
                print(f"✅ {model_name} 下载成功")
                return True
            else:
                print(f"❌ {model_name} 下载失败：文件太小")
                return False
        else:
            print(f"❌ {model_name} 下载失败：{result.stderr}")
            return False
    except Exception as e:
        print(f"❌ {model_name} 下载失败：{str(e)}")
        return False

def download_all_models(selected_models=None):
    """下载所有或指定的模型"""
    if selected_models is None:
        selected_models = models.keys()
    
    success_count = 0
    total_count = len(selected_models)
    
    for model_name in selected_models:
        if model_name in models:
            success = download_model(model_name, models[model_name])
            if success:
                success_count += 1
        else:
            print(f"❌ 未知模型: {model_name}")
    
    print(f"\n📊 下载完成: {success_count}/{total_count} 个模型下载成功")
    return success_count == total_count

In [8]:
# 推荐下载的核心模型（覆盖主要任务）
essential_models = [
    "003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth",  # 真实世界超分辨率
    "001_classicalSR_DF2K_s64w8_SwinIR-M_x4.pth",        # 经典超分辨率4x
    "005_colorDN_DFWB_s128w8_SwinIR-M_noise25.pth",      # 彩色图像去噪
    "006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg30.pth",      # 彩色JPEG伪影减少
]

print("🎯 推荐下载核心模型（适合快速测试）:")
for i, model in enumerate(essential_models, 1):
    print(f"{i}. {model}")

print("\n选择下载方式:")
print("1. 下载核心模型（推荐，约500MB）")
print("2. 下载所有模型（约3GB+）")
print("3. 自定义选择模型")

# 这里可以根据需要修改选择
download_choice = 1  # 修改这个数字来选择下载方式

if download_choice == 1:
    print("\n📥 开始下载核心模型...")
    download_all_models(essential_models)
elif download_choice == 2:
    print("\n📥 开始下载所有模型...")
    download_all_models()
else:
    print("\n请手动选择要下载的模型")

🎯 推荐下载核心模型（适合快速测试）:
1. 003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth
2. 001_classicalSR_DF2K_s64w8_SwinIR-M_x4.pth
3. 005_colorDN_DFWB_s128w8_SwinIR-M_noise25.pth
4. 006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg30.pth

选择下载方式:
1. 下载核心模型（推荐，约500MB）
2. 下载所有模型（约3GB+）
3. 自定义选择模型

📥 开始下载核心模型...
✅ 003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth 已存在，跳过下载
✅ 001_classicalSR_DF2K_s64w8_SwinIR-M_x4.pth 已存在，跳过下载
✅ 005_colorDN_DFWB_s128w8_SwinIR-M_noise25.pth 已存在，跳过下载
✅ 006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg30.pth 已存在，跳过下载

📊 下载完成: 4/4 个模型下载成功


## 3. 下载测试数据集

下载一些测试图像来验证模型效果。

In [9]:
# 检查现有的测试集
test_dirs = ['testsets/Set5', 'testsets/Set12', 'testsets/classic5', 'testsets/McMaster', 'testsets/RealSRSet+5images']
existing_datasets = []

for test_dir in test_dirs:
    if os.path.exists(test_dir):
        image_count = len([f for f in os.listdir(test_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif'))])
        if image_count > 0:
            existing_datasets.append((test_dir, image_count))
            print(f"✅ {test_dir}: {image_count} 张图像")

if existing_datasets:
    print(f"\n已有 {len(existing_datasets)} 个测试数据集可用")
else:
    print("❌ 没有找到测试数据集")
    
# 下载RealSRSet测试集（如果不存在）
realsr_url = "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/RealSRSet+5images.zip"
if not os.path.exists('testsets/RealSRSet+5images'):
    print("\n📥 下载 RealSRSet+5images 测试集...")
    try:
        subprocess.run(f"wget --no-check-certificate -O RealSRSet.zip {realsr_url}", shell=True, check=True)
        subprocess.run("unzip -q RealSRSet.zip -d testsets/", shell=True, check=True)
        subprocess.run("rm RealSRSet.zip", shell=True)
        print("✅ RealSRSet+5images 下载完成")
    except:
        print("❌ RealSRSet+5images 下载失败")

print("\n📊 测试数据集状态:")
for test_dir in test_dirs:
    if os.path.exists(test_dir):
        image_count = len([f for f in os.listdir(test_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif'))])
        print(f"  {test_dir}: {image_count} 张图像")
    else:
        print(f"  {test_dir}: 不存在")

✅ testsets/Set12: 12 张图像
✅ testsets/classic5: 5 张图像
✅ testsets/McMaster: 18 张图像
✅ testsets/RealSRSet+5images: 25 张图像

已有 4 个测试数据集可用

📊 测试数据集状态:
  testsets/Set5: 0 张图像
  testsets/Set12: 12 张图像
  testsets/classic5: 5 张图像
  testsets/McMaster: 18 张图像
  testsets/RealSRSet+5images: 25 张图像


## 4. 图像超分辨率实现

实现使用SwinIR进行图像超分辨率的功能。

In [10]:
def run_super_resolution(task='real_sr', scale=4, input_path='testsets/RealSRSet+5images', 
                        model_path=None, tile_size=400, output_dir=None):
    """
    运行图像超分辨率
    
    Args:
        task: 'classical_sr', 'lightweight_sr', 'real_sr'
        scale: 超分辨率倍数 (2, 3, 4, 8)
        input_path: 输入图像路径或文件夹
        model_path: 模型路径（如果为None则自动选择）
        tile_size: 分块大小（处理大图像时使用）
        output_dir: 输出目录
    """
    
    # 自动选择模型
    if model_path is None:
        if task == 'real_sr':
            model_path = 'model_zoo/swinir/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth'
        elif task == 'classical_sr':
            model_path = f'model_zoo/swinir/001_classicalSR_DF2K_s64w8_SwinIR-M_x{scale}.pth'
        elif task == 'lightweight_sr':
            model_path = f'model_zoo/swinir/002_lightweightSR_DIV2K_s64w8_SwinIR-S_x{scale}.pth'
    
    # 检查模型是否存在
    if not os.path.exists(model_path):
        print(f"❌ 模型文件不存在: {model_path}")
        print("请先下载对应的模型文件")
        return False
    
    # 构建命令
    if task == 'real_sr':
        cmd = f"python main_test_swinir.py --task {task} --scale {scale} --model_path {model_path} --folder_lq {input_path}"
    else:
        cmd = f"python main_test_swinir.py --task {task} --scale {scale} --model_path {model_path} --folder_gt {input_path}"
    
    # 添加分块参数
    if tile_size:
        cmd += f" --tile {tile_size}"
    
    print(f"🚀 运行命令: {cmd}")
    
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=300)
        if result.returncode == 0:
            print("✅ 超分辨率处理完成")
            print(result.stdout)
            return True
        else:
            print("❌ 处理失败")
            print(result.stderr)
            return False
    except subprocess.TimeoutExpired:
        print("❌ 处理超时")
        return False
    except Exception as e:
        print(f"❌ 处理出错: {str(e)}")
        return False

# 测试真实世界图像超分辨率
print("🎯 测试真实世界图像超分辨率 (4x)...")
if os.path.exists('testsets/RealSRSet+5images'):
    success = run_super_resolution(task='real_sr', scale=4, 
                                 input_path='testsets/RealSRSet+5images',
                                 tile_size=400)
    if success:
        print("📁 结果保存在: results/swinir_real_sr_x4/")
else:
    print("❌ 测试图像不存在，请先下载测试数据集")

🎯 测试真实世界图像超分辨率 (4x)...
🚀 运行命令: python main_test_swinir.py --task real_sr --scale 4 --model_path model_zoo/swinir/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth --folder_lq testsets/RealSRSet+5images --tile 400
✅ 超分辨率处理完成
loading model from model_zoo/swinir/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth
Testing 0 00003               
Testing 1 0014                
Testing 2 0030                
Testing 3 ADE_val_00000114    
Testing 4 Lincoln             
Testing 5 OST_009             
Testing 6 building            
Testing 7 butterfly           
Testing 8 butterfly2          
Testing 9 chip                
Testing 10 comic1              
Testing 11 comic2              
Testing 12 comic3              
Testing 13 computer            
Testing 14 dog                 
Testing 15 dped_crop00061      
Testing 16 foreman             
Testing 17 frog                
Testing 18 oldphoto2           
Testing 19 oldphoto3           
Testing 20 oldphoto6           
Testing 21 painting            


## 5. 图像去噪实现

实现使用SwinIR进行图像去噪的功能。

In [11]:
def run_denoising(task='color_dn', noise_level=25, input_path='testsets/McMaster', 
                  model_path=None):
    """
    运行图像去噪
    
    Args:
        task: 'gray_dn' 或 'color_dn'
        noise_level: 噪声水平 (15, 25, 50)
        input_path: 输入图像路径或文件夹
        model_path: 模型路径（如果为None则自动选择）
    """
    
    # 自动选择模型
    if model_path is None:
        if task == 'color_dn':
            model_path = f'model_zoo/swinir/005_colorDN_DFWB_s128w8_SwinIR-M_noise{noise_level}.pth'
        elif task == 'gray_dn':
            model_path = f'model_zoo/swinir/004_grayDN_DFWB_s128w8_SwinIR-M_noise{noise_level}.pth'
    
    # 检查模型是否存在
    if not os.path.exists(model_path):
        print(f"❌ 模型文件不存在: {model_path}")
        print("请先下载对应的模型文件")
        return False
    
    # 构建命令
    cmd = f"python main_test_swinir.py --task {task} --noise {noise_level} --model_path {model_path} --folder_gt {input_path}"
    
    print(f"🚀 运行命令: {cmd}")
    
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=300)
        if result.returncode == 0:
            print("✅ 图像去噪处理完成")
            print(result.stdout)
            return True
        else:
            print("❌ 处理失败")
            print(result.stderr)
            return False
    except subprocess.TimeoutExpired:
        print("❌ 处理超时")
        return False
    except Exception as e:
        print(f"❌ 处理出错: {str(e)}")
        return False

# 测试彩色图像去噪
print("🎯 测试彩色图像去噪 (噪声水平25)...")
if os.path.exists('testsets/McMaster'):
    success = run_denoising(task='color_dn', noise_level=25, 
                          input_path='testsets/McMaster')
    if success:
        print("📁 结果保存在: results/swinir_color_dn_noise25/")
else:
    print("❌ 测试图像不存在，使用现有图像测试")
    # 如果McMaster不存在，尝试使用其他测试集
    for test_dir in ['testsets/Set5', 'testsets/Set12', 'testsets/classic5']:
        if os.path.exists(test_dir):
            print(f"📁 使用 {test_dir} 进行测试")
            run_denoising(task='color_dn', noise_level=25, input_path=test_dir)
            break

🎯 测试彩色图像去噪 (噪声水平25)...
🚀 运行命令: python main_test_swinir.py --task color_dn --noise 25 --model_path model_zoo/swinir/005_colorDN_DFWB_s128w8_SwinIR-M_noise25.pth --folder_gt testsets/McMaster
❌ 处理失败
/root/miniconda3/lib/python3.8/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/root/miniconda3/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Traceback (most recent call last):
  File "main_test_swinir.py", line 309, in <module>
    main()
  File "main_test_swinir.py", line 44, in main
    model = define_model(args)
  File "main_test_swinir.py",

## 6. JPEG压缩伪影减少

实现使用SwinIR去除JPEG压缩伪影的功能。

In [ ]:
def run_jpeg_artifact_reduction(task='color_jpeg_car', jpeg_quality=30, 
                               input_path='testsets/classic5', model_path=None):
    """
    运行JPEG压缩伪影减少
    
    Args:
        task: 'jpeg_car' 或 'color_jpeg_car'
        jpeg_quality: JPEG质量水平 (10, 20, 30, 40)
        input_path: 输入图像路径或文件夹
        model_path: 模型路径（如果为None则自动选择）
    """
    
    # 自动选择模型
    if model_path is None:
        if task == 'color_jpeg_car':
            model_path = f'model_zoo/swinir/006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg{jpeg_quality}.pth'
        elif task == 'jpeg_car':
            model_path = f'model_zoo/swinir/006_CAR_DFWB_s126w7_SwinIR-M_jpeg{jpeg_quality}.pth'
    
    # 检查模型是否存在
    if not os.path.exists(model_path):
        print(f"❌ 模型文件不存在: {model_path}")
        print("请先下载对应的模型文件")
        return False
    
    # 构建命令
    cmd = f"python main_test_swinir.py --task {task} --jpeg {jpeg_quality} --model_path {model_path} --folder_gt {input_path}"
    
    print(f"🚀 运行命令: {cmd}")
    
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=300)
        if result.returncode == 0:
            print("✅ JPEG伪影减少处理完成")
            print(result.stdout)
            return True
        else:
            print("❌ 处理失败")
            print(result.stderr)
            return False
    except subprocess.TimeoutExpired:
        print("❌ 处理超时")
        return False
    except Exception as e:
        print(f"❌ 处理出错: {str(e)}")
        return False

# 测试彩色JPEG压缩伪影减少
print("🎯 测试彩色JPEG压缩伪影减少 (质量30)...")
if os.path.exists('testsets/classic5'):
    success = run_jpeg_artifact_reduction(task='color_jpeg_car', jpeg_quality=30, 
                                        input_path='testsets/classic5')
    if success:
        print("📁 结果保存在: results/swinir_color_jpeg_car_jpeg30/")
else:
    print("❌ 测试图像不存在，使用现有图像测试")
    # 如果classic5不存在，尝试使用其他测试集
    for test_dir in ['testsets/Set5', 'testsets/Set12', 'testsets/McMaster']:
        if os.path.exists(test_dir):
            print(f"📁 使用 {test_dir} 进行测试")
            run_jpeg_artifact_reduction(task='color_jpeg_car', jpeg_quality=30, 
                                       input_path=test_dir)
            break

## 7. 批量处理功能

实现批量处理多张图像的功能。

In [ ]:
def batch_process_images(input_folder, output_folder, task='real_sr', **kwargs):
    """
    批量处理图像
    
    Args:
        input_folder: 输入文件夹路径
        output_folder: 输出文件夹路径
        task: 处理任务类型
        **kwargs: 其他参数
    """
    
    if not os.path.exists(input_folder):
        print(f"❌ 输入文件夹不存在: {input_folder}")
        return False
    
    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)
    
    # 获取图像文件列表
    image_extensions = ['.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff']
    image_files = []
    
    for ext in image_extensions:
        image_files.extend(Path(input_folder).glob(f'*{ext}'))
        image_files.extend(Path(input_folder).glob(f'*{ext.upper()}'))
    
    if not image_files:
        print(f"❌ 在 {input_folder} 中没有找到图像文件")
        return False
    
    print(f"📁 找到 {len(image_files)} 张图像，开始批量处理...")
    
    success_count = 0
    
    for i, image_file in enumerate(image_files, 1):
        print(f"\n🔄 处理第 {i}/{len(image_files)} 张图像: {image_file.name}")
        
        # 创建临时输入文件夹
        temp_input = f"temp_input_{i}"
        os.makedirs(temp_input, exist_ok=True)
        
        try:
            # 复制图像到临时文件夹
            import shutil
            shutil.copy2(image_file, temp_input)
            
            # 根据任务类型调用相应的处理函数
            if task in ['real_sr', 'classical_sr', 'lightweight_sr']:
                success = run_super_resolution(
                    task=task, 
                    input_path=temp_input,
                    **kwargs
                )
            elif task in ['color_dn', 'gray_dn']:
                success = run_denoising(
                    task=task,
                    input_path=temp_input,
                    **kwargs
                )
            elif task in ['color_jpeg_car', 'jpeg_car']:
                success = run_jpeg_artifact_reduction(
                    task=task,
                    input_path=temp_input,
                    **kwargs
                )
            else:
                print(f"❌ 不支持的任务类型: {task}")
                success = False
            
            if success:
                success_count += 1
                print(f"✅ {image_file.name} 处理成功")
            else:
                print(f"❌ {image_file.name} 处理失败")
                
        finally:
            # 清理临时文件夹
            if os.path.exists(temp_input):
                shutil.rmtree(temp_input)
    
    print(f"\n📊 批量处理完成: {success_count}/{len(image_files)} 张图像处理成功")
    return success_count == len(image_files)

# 示例：批量处理真实世界超分辨率
print("💡 批量处理示例：")
print("1. 将要处理的图像放入指定文件夹")
print("2. 调用 batch_process_images() 函数")
print("3. 结果会保存到对应的输出文件夹")
print()
print("示例代码：")
print("batch_process_images('my_images', 'results/my_sr_results', task='real_sr', scale=4)")

## 8. 结果可视化和对比

创建可视化功能来对比处理前后的图像效果。

In [ ]:
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

def display_results(result_folder, original_folder=None, max_images=5):
    """
    显示处理结果
    
    Args:
        result_folder: 结果文件夹路径
        original_folder: 原始图像文件夹路径（可选）
        max_images: 最多显示的图像数量
    """
    
    if not os.path.exists(result_folder):
        print(f"❌ 结果文件夹不存在: {result_folder}")
        return
    
    # 获取结果图像列表
    result_files = []
    for ext in ['.png', '.jpg', '.jpeg']:
        result_files.extend(Path(result_folder).glob(f'*{ext}'))
    
    if not result_files:
        print(f"❌ 在 {result_folder} 中没有找到图像")
        return
    
    result_files = sorted(result_files)[:max_images]
    
    # 设置图像显示
    fig_width = 15
    fig_height = 5 * len(result_files)
    
    if original_folder and os.path.exists(original_folder):
        fig, axes = plt.subplots(len(result_files), 2, figsize=(fig_width, fig_height))
        if len(result_files) == 1:
            axes = axes.reshape(1, -1)
    else:
        fig, axes = plt.subplots(len(result_files), 1, figsize=(fig_width//2, fig_height))
        if len(result_files) == 1:
            axes = [axes]
    
    for i, result_file in enumerate(result_files):
        # 显示处理后的图像
        result_img = plt.imread(result_file)
        
        if original_folder and os.path.exists(original_folder):
            # 尝试找到对应的原始图像
            original_name = result_file.stem.replace('_SwinIR', '')
            original_file = None
            
            for ext in ['.png', '.jpg', '.jpeg', '.bmp', '.tif']:
                potential_file = Path(original_folder) / f"{original_name}{ext}"
                if potential_file.exists():
                    original_file = potential_file
                    break
            
            if original_file:
                original_img = plt.imread(original_file)
                
                # 显示原始图像
                axes[i, 0].imshow(original_img)
                axes[i, 0].set_title(f'Original: {original_file.name}')
                axes[i, 0].axis('off')
                
                # 显示处理后图像
                axes[i, 1].imshow(result_img)
                axes[i, 1].set_title(f'SwinIR Result: {result_file.name}')
                axes[i, 1].axis('off')
                
                # 计算质量指标（如果尺寸匹配）
                if original_img.shape == result_img.shape:
                    try:
                        psnr_val = psnr(original_img, result_img)
                        ssim_val = ssim(original_img, result_img, multichannel=True, channel_axis=-1)
                        axes[i, 1].set_xlabel(f'PSNR: {psnr_val:.2f}dB, SSIM: {ssim_val:.4f}')
                    except:
                        pass
            else:
                # 只显示结果图像
                axes[i, 1].imshow(result_img)
                axes[i, 1].set_title(f'SwinIR Result: {result_file.name}')
                axes[i, 1].axis('off')
                axes[i, 0].axis('off')
        else:
            # 只显示结果图像
            axes[i].imshow(result_img)
            axes[i].set_title(f'SwinIR Result: {result_file.name}')
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

def compare_all_results():
    """比较所有可用的结果"""
    
    result_folders = []
    
    # 查找所有结果文件夹
    if os.path.exists('results'):
        for item in os.listdir('results'):
            result_path = os.path.join('results', item)
            if os.path.isdir(result_path):
                # 检查是否有图像文件
                has_images = any(
                    Path(result_path).glob(f'*.{ext}') 
                    for ext in ['png', 'jpg', 'jpeg']
                )
                if has_images:
                    result_folders.append(result_path)
    
    if not result_folders:
        print("❌ 没有找到任何结果文件夹")
        return
    
    print(f"📊 找到 {len(result_folders)} 个结果文件夹:")
    for i, folder in enumerate(result_folders, 1):
        image_count = len([f for f in os.listdir(folder) 
                          if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        print(f"{i}. {folder}: {image_count} 张图像")
    
    # 显示每个文件夹的结果
    for folder in result_folders:
        print(f"\n📁 显示 {folder} 的结果:")
        display_results(folder, max_images=3)

# 查看现有结果
print("🎯 查看处理结果:")
compare_all_results()

## 总结和快速命令

### 📋 使用步骤总结：

1. **环境设置** - 检查GPU和依赖
2. **下载模型** - 选择需要的预训练模型
3. **下载数据** - 获取测试图像
4. **运行推理** - 选择任务类型进行处理
5. **查看结果** - 可视化处理效果

### 🚀 快速命令参考：

```bash
# 真实世界图像超分辨率
python main_test_swinir.py --task real_sr --scale 4 --model_path model_zoo/swinir/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth --folder_lq testsets/RealSRSet+5images --tile 400

# 彩色图像去噪
python main_test_swinir.py --task color_dn --noise 25 --model_path model_zoo/swinir/005_colorDN_DFWB_s128w8_SwinIR-M_noise25.pth --folder_gt testsets/McMaster

# 彩色JPEG伪影减少
python main_test_swinir.py --task color_jpeg_car --jpeg 30 --model_path model_zoo/swinir/006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg30.pth --folder_gt testsets/classic5
```

### 💡 提示：
- 使用 `--tile 400` 参数处理大图像
- 结果保存在 `results/` 文件夹下
- 可以修改 `download_choice` 变量来选择下载哪些模型
- 所有函数都支持自定义参数

---

**🎉 现在你可以开始使用SwinIR进行图像复原了！**

## 9. 检查现有模型和执行所有任务

检查你现在有哪些模型，然后测试所有可能的任务。

In [ ]:
def check_models_and_tasks():
    """检查现有模型和可执行的任务"""
    
    print("🔍 检查现有模型文件...")
    model_dir = "model_zoo/swinir"
    
    if not os.path.exists(model_dir):
        print(f"❌ 模型目录不存在: {model_dir}")
        return
    
    # 获取所有模型文件
    existing_models = []
    for file in os.listdir(model_dir):
        if file.endswith('.pth'):
            file_path = os.path.join(model_dir, file)
            file_size = os.path.getsize(file_path) / (1024*1024)  # MB
            existing_models.append((file, file_size))
    
    if not existing_models:
        print("❌ 没有找到任何模型文件")
        return
    
    print(f"\n📊 找到 {len(existing_models)} 个模型文件:")
    for model, size in existing_models:
        print(f"  ✅ {model} ({size:.1f} MB)")
    
    # 检查测试数据集
    print(f"\n🔍 检查测试数据集...")
    test_dirs = ['testsets/Set5', 'testsets/Set12', 'testsets/classic5', 
                 'testsets/McMaster', 'testsets/RealSRSet+5images']
    
    available_datasets = []
    for test_dir in test_dirs:
        if os.path.exists(test_dir):
            image_count = len([f for f in os.listdir(test_dir) 
                              if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif'))])
            if image_count > 0:
                available_datasets.append((test_dir, image_count))
    
    print(f"📊 可用测试数据集:")
    for dataset, count in available_datasets:
        print(f"  ✅ {dataset}: {count} 张图像")
    
    # 任务与模型映射
    task_model_mapping = {
        'real_sr': ['003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth',
                   '003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth'],
        'classical_sr_x2': ['001_classicalSR_DIV2K_s48w8_SwinIR-M_x2.pth',
                            '001_classicalSR_DF2K_s64w8_SwinIR-M_x2.pth'],
        'classical_sr_x4': ['001_classicalSR_DIV2K_s48w8_SwinIR-M_x4.pth',
                            '001_classicalSR_DF2K_s64w8_SwinIR-M_x4.pth'],
        'lightweight_sr_x4': ['002_lightweightSR_DIV2K_s64w8_SwinIR-S_x4.pth'],
        'color_dn_25': ['005_colorDN_DFWB_s128w8_SwinIR-M_noise25.pth'],
        'gray_dn_25': ['004_grayDN_DFWB_s128w8_SwinIR-M_noise25.pth'],
        'color_jpeg_car_30': ['006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg30.pth'],
        'jpeg_car_30': ['006_CAR_DFWB_s126w7_SwinIR-M_jpeg30.pth']
    }
    
    # 检查可执行的任务
    print(f"\n🎯 可执行的任务:")
    executable_tasks = []
    
    for task, required_models in task_model_mapping.items():
        has_model = any(model in [m[0] for m in existing_models] for model in required_models)
        if has_model:
            available_model = next(model for model in required_models 
                                 if model in [m[0] for m in existing_models])
            executable_tasks.append((task, available_model))
            print(f"  ✅ {task}: {available_model}")
        else:
            print(f"  ❌ {task}: 缺少模型 {required_models}")
    
    return executable_tasks, available_datasets

# 执行检查
executable_tasks, available_datasets = check_models_and_tasks()

In [ ]:
def execute_all_available_tasks():
    """执行所有可用的任务"""
    
    executable_tasks, available_datasets = check_models_and_tasks()
    
    if not executable_tasks:
        print("❌ 没有可执行的任务，请先下载模型")
        return
    
    if not available_datasets:
        print("❌ 没有可用的测试数据集")
        return
    
    # 选择一个测试数据集（优先选择Set5，因为通用性好）
    test_dataset = None
    for dataset, count in available_datasets:
        if 'Set5' in dataset:
            test_dataset = dataset
            break
    if not test_dataset:
        test_dataset = available_datasets[0][0]  # 使用第一个可用数据集
    
    print(f"\n🚀 开始执行所有可用任务，使用数据集: {test_dataset}")
    
    results = []
    
    for task, model_file in executable_tasks:
        print(f"\n{'='*60}")
        print(f"🎯 执行任务: {task}")
        print(f"📦 使用模型: {model_file}")
        print(f"📁 测试数据: {test_dataset}")
        
        model_path = f"model_zoo/swinir/{model_file}"
        
        try:
            # 根据任务类型执行
            if 'real_sr' in task:
                success = run_super_resolution(
                    task='real_sr', 
                    scale=4, 
                    input_path=test_dataset,
                    model_path=model_path,
                    tile_size=400
                )
                
            elif 'classical_sr' in task:
                # 经典超分辨率需要特殊处理
                scale = 2 if 'x2' in task else 4
                training_patch_size = 48 if 'DIV2K_s48' in model_file else 64
                
                # 构建特殊命令
                cmd = f"python main_test_swinir.py --task classical_sr --scale {scale} --training_patch_size {training_patch_size} --model_path {model_path} --folder_gt {test_dataset}"
                
                print(f"🚀 运行命令: {cmd}")
                result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=300)
                success = result.returncode == 0
                
                if success:
                    print("✅ 经典超分辨率处理完成")
                else:
                    print(f"❌ 处理失败: {result.stderr}")
                    
            elif 'lightweight_sr' in task:
                success = run_super_resolution(
                    task='lightweight_sr', 
                    scale=4, 
                    input_path=test_dataset,
                    model_path=model_path
                )
                
            elif 'color_dn' in task:
                success = run_denoising(
                    task='color_dn', 
                    noise_level=25, 
                    input_path=test_dataset,
                    model_path=model_path
                )
                
            elif 'gray_dn' in task:
                success = run_denoising(
                    task='gray_dn', 
                    noise_level=25, 
                    input_path=test_dataset,
                    model_path=model_path
                )
                
            elif 'color_jpeg_car' in task:
                success = run_jpeg_artifact_reduction(
                    task='color_jpeg_car', 
                    jpeg_quality=30, 
                    input_path=test_dataset,
                    model_path=model_path
                )
                
            elif 'jpeg_car' in task:
                success = run_jpeg_artifact_reduction(
                    task='jpeg_car', 
                    jpeg_quality=30, 
                    input_path=test_dataset,
                    model_path=model_path
                )
                
            else:
                print(f"❌ 未知任务类型: {task}")
                success = False
            
            results.append((task, success))
            
            if success:
                print(f"✅ {task} 执行成功")
            else:
                print(f"❌ {task} 执行失败")
                
        except Exception as e:
            print(f"❌ {task} 执行出错: {str(e)}")
            results.append((task, False))
    
    # 总结结果
    print(f"\n{'='*60}")
    print("📊 执行结果总结:")
    successful_tasks = [task for task, success in results if success]
    failed_tasks = [task for task, success in results if not success]
    
    print(f"✅ 成功执行: {len(successful_tasks)}/{len(results)} 个任务")
    for task in successful_tasks:
        print(f"  ✅ {task}")
    
    if failed_tasks:
        print(f"❌ 执行失败: {len(failed_tasks)} 个任务")
        for task in failed_tasks:
            print(f"  ❌ {task}")
    
    # 显示结果文件夹
    print(f"\n📁 查看结果文件夹:")
    if os.path.exists('results'):
        for item in os.listdir('results'):
            result_path = os.path.join('results', item)
            if os.path.isdir(result_path):
                image_count = len([f for f in os.listdir(result_path) 
                                  if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
                if image_count > 0:
                    print(f"  📁 {result_path}: {image_count} 张图像")

# 执行所有可用任务
print("🎯 开始执行所有可用任务...")
execute_all_available_tasks()